In [ ]:
%pip install langchain
%pip install pinecone-client
%pip install pypdf
%pip install langchain-text-splitters

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
HUGGING_FACE_API = os.getenv('HUGGING_FACE_API')

In [5]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("pdfs/")
data = loader.load()

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    
    chunk_size=500,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

text_chunks = text_splitter.split_documents(data)

In [10]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=HUGGING_FACE_API, model_name="sentence-transformers/all-MiniLM-l6-v2"
)